In [8]:
# ==========================================
# 1. IMPORTS
# ==========================================
# We import the necessary tools to build our AI team.
# 'Agent': The digital employee.
# 'Task': The specific job the employee must do.
# 'Crew': The manager that coordinates the agents and tasks.
from crewai import Agent, Task, Crew
from dotenv import load_dotenv
load_dotenv()

# We import ChatOpenAI to allow our agents to talk to OpenAI's GPT models.
from langchain_openai import ChatOpenAI

# We import 'os' to manage environment variables (like secret passwords/API keys).
import os

# NOTE: Before running this, you usually need to set your OpenAI API key.
# os.environ["OPENAI_API_KEY"] = "sk-..."

# ==========================================
# 2. THE BRAIN (LLM)
# ==========================================
# This defines the "brain" our agent will use to think.
# We are using GPT-4o-mini, which is a fast and cost-effective model.
llm = ChatOpenAI(model="gpt-4o-mini")

# ==========================================
# 3. DEFINE THE AGENT (The Employee)
# ==========================================
# Think of this as writing a job description for a new hire.
fetcher = Agent(
    # 'role': The job title. This tells the AI what kind of expert it is.
    role="ResearchFetcher",
    
    # 'goal': The specific objective this agent wants to achieve. 
    # The AI uses this to measure if it has done a good job.
    goal="Find relevant academic papers on a topic and summarize abstracts",
    
    # 'backstory': This provides context and personality. 
    # It helps the AI 'get into character' to provide better answers.
    backstory=(
        "You are an expert research assistant familiar with arXiv and Semantic Scholar. "
        "You know how to search for papers and read their abstracts."
    ),
    
    # 'allow_delegation': If True, this agent could ask *other* agents for help.
    # Since this is a solo mission, we set it to False.
    allow_delegation=False,
    
    # We assign the specific brain (LLM) we defined earlier to this agent.
    llm=llm,
)

# ==========================================
# 4. DEFINE THE TASK (The Assignment)
# ==========================================
# This is the actual item on the "To-Do" list for our employee.
fetch_task = Task(
    # 'description': Detailed instructions on what to do. Be specific here!
    description="Search for 3 recent papers on 'large language model alignment' and summarize each in 3–4 sentences.",
    
    # 'expected_output': Tells the agent exactly what the final result should look like.
    # This prevents the AI from rambling or giving the wrong format.
    expected_output="List of papers with title, authors, and summarized abstract.",
    
    # 'agent': We explicitly assign this task to our 'fetcher' agent.
    agent=fetcher,
)

# ==========================================
# 5. DEFINE THE CREW (The Team)
# ==========================================
# The Crew is the container that holds everything together.
crew = Crew(
    agents=[fetcher],  # A list of all agents involved in this workflow.
    tasks=[fetch_task] # A list of all tasks to be completed.
)

# ==========================================
# 6. EXECUTION (Start the Work)
# ==========================================
# 'kickoff()' is the start button. It triggers the agents to begin working on their tasks.
print("Starting the research crew...")
result = crew.kickoff()

# Finally, we print the result of the agent's work to the console.
print("######################")
print(result)

Starting the research crew...
######################
1. Title: "Scaling Laws for Neural Language Models"  
   Authors: Jared Kaplan, Sam McCandlish, Tom Henighan, et al.  
   Abstract: This paper investigates the scaling laws of neural language models, showing how performance improves predictively with model size, dataset size, and training compute. The authors derive a formula for relationship between the power of language models and scaling. They demonstrate that larger models are likely to perform better on alignment tasks, indicating a potential path toward achieving alignment with human-like behaviors and preferences.

2. Title: "On the Importance of Alignment for AI Systems"  
   Authors: Stuart Russell, Daniel Dewey, Max Tegmark  
   Abstract: The authors emphasize the critical necessity for aligning AI systems, particularly large language models, with human values and intentions. They outline various methodologies that can be employed to improve alignment, such as interpretabil

In [1]:
# 01_single_agent.py
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

load_dotenv()

# Initialize the search tool
search_tool = SerperDevTool()

# Define the agent
researcher = Agent(
    role="Senior Research Analyst",
    goal="Find and analyze the most relevant academic papers on a given topic",
    backstory="""You are an experienced academic researcher with deep expertise 
    in finding relevant literature. You prioritize peer-reviewed sources from 
    reputable venues like arXiv, NeurIPS, ICML, and ACL. You focus on recent 
    publications (last 3 years) unless older seminal works are essential. 
    You're thorough but efficient—you know when you've found enough.""",
    tools=[search_tool],
    verbose=True
)

# Define the task
research_task = Task(
    description="""Search for the top 3 most relevant academic papers on: {topic}
    
    For each paper, provide:
    1. Title
    2. Authors (first author et al. is fine for papers with many authors)
    3. Publication year and venue
    4. Key findings (2-3 sentences summarizing the main contribution)
    5. Relevance (1 sentence on why this paper matters for the topic)
    
    Focus on influential papers with significant citations or from top venues.
    Prioritize recent work unless older papers are foundational.
    """,
    expected_output="""A structured markdown list of exactly 3 papers, 
    each containing: title, authors, year/venue, key findings, and relevance.""",
    agent=researcher
)

# Create and run the crew
crew = Crew(
    agents=[researcher],
    tasks=[research_task],
    process=Process.sequential,
    verbose=True
)

if __name__ == "__main__":
    topic = "large language model alignment"
    print(f"\n🔍 Researching: {topic}\n")
    
    result = crew.kickoff(inputs={"topic": topic})
    
    print("\n" + "="*60)
    print("RESEARCH RESULTS")
    print("="*60)
    print(result.raw)


🔍 Researching: large language model alignment



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 27f33c56-2a38-4a98-8a94-2093ea025e65                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Task: Search for the top 3 most relevant academic papers on: large language model alignment                    │
│                                                                                                                 │
│      For each paper, provide:                                                                                   │
│      1. Title                                                                                                   │
│      2. Authors (first author et al. is fine for papers with many authors)                                      │
│      3. Publication year and venue                                                                              │
│      4. Key findings (2-3 sentences summarizing the main contribution)                                          │
│      5. Relevance (1 sentence on why this paper matters for the topic)                                          │
│                                                                                                                 │
│      Focus on influential papers with significant citations or from top venues.                                 │
│      Prioritize recent work unless older papers are foundational.                                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Thought: I need to search for recent academic papers specifically focused on the topic of "large language      │
│  model alignment" to identify influential works.                                                                │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1. **Title**: [Large Language Model Alignment: A Survey](https://arxiv.org/abs/2309.15025)                     │
│     **Authors**: T. Shen et al.                                                                                 │
│     **Year/Venue**: 2023, arXiv                                                                                 │
│     **Key Findings**: This survey provides a comprehensive investigation of various alignment methodologies     │
│  aimed at large language models (LLMs) and classifies them into distinct categories based on their theoretical  │
│  frameworks and practical applications. The authors emphasize the significance of aligning LLMs with human      │
│  values and decision-making processes.                                                                          │
│     **Relevance**: This paper is crucial as it offers an extensive overview of alignment techniques which is    │
│  vital for understanding how to ensure LLMs act in accordance with human intentions.                            │
│                                                                                                                 │
│  2. **Title**: [Democratizing Large Language Model                                                              │
│  Alignment](https://proceedings.neurips.cc/paper_files/paper/2023/hash/949f0f8f32267d297c2d4e3ee10a2e7e-Abstra  │
│  ct-Datasets_and_Benchmarks.html)                                                                               │
│     **Authors**: A. Köpf et al.                                                                                 │
│     **Year/Venue**: 2023, NeurIPS                                                                               │
│     **Key Findings**: This paper discusses the challenges related to aligning large language models with        │
│  diverse human preferences, highlighting the importance of usability and its effect on rapid adoption of LLMs.  │
│  The authors propose methods to democratize access and engagement with LLMs to improve alignment outcomes.      │
│     **Relevance**: This work matters as it emphasizes the societal impact of LLM alignment and strives to make  │
│  alignment accessible and equitable for a broader audience.                                                     │
│                                                                                                                 │
│  3. **Title**: [A Comprehensive Survey of LLM Alignment Techniques](https://arxiv.org/abs/2407.16216)           │
│     **Authors**: Z. Wang et al.                                                                                 │
│     **Year/Venue**: 2024, arXiv                                                                                 │
│     **Key Findings**: This survey categorizes the existing literature on LLM alignment into several key topics  │
│  and provides detailed elaborations on each method's strengths and weaknesses. The authors aim to bridge the    │
│  gaps in current research by synthesizing findings from various sources and guiding future research             │
│  directions.                                                                                                    │
│     **Relevance**: This paper is significant because it compiles and clarifies the diverse approaches to LLM    │
│  alignment, setting the stage for future advancements i

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 1c912b45-8ad7-473a-926d-11f450899ecd                                                                     │
│  Agent: Senior Research Analyst                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 27f33c56-2a38-4a98-8a94-2093ea025e65                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: 1. **Title**: [Large Language Model Alignment: A Survey](https://arxiv.org/abs/2309.15025)       │
│     **Authors**: T. Shen et al.                                                                                 │
│     **Year/Venue**: 2023, arXiv                                                                                 │
│     **Key Findings**: This survey provides a comprehensive investigation of various alignment methodologies     │
│  aimed at large language models (LLMs) and classifies them into distinct categories based on their theoretical  │
│  frameworks and practical applications. The authors emphasize the significance of aligning LLMs with human      │
│  values and decision-making processes.                                                                          │
│     **Relevance**: This paper is crucial as it offers an extensive overview of alignment techniques which is    │
│  vital for understanding how to ensure LLMs act in accordance with human intentions.                            │
│                                                                                                                 │
│  2. **Title**: [Democratizing Large Language Model                                                              │
│  Alignment](https://proceedings.neurips.cc/paper_files/paper/2023/hash/949f0f8f32267d297c2d4e3ee10a2e7e-Abstra  │
│  ct-Datasets_and_Benchmarks.html)                                                                               │
│     **Authors**: A. Köpf et al.                                                                                 │
│     **Year/Venue**: 2023, NeurIPS                                                                               │
│     **Key Findings**: This paper discusses the challenges related to aligning large language models with        │
│  diverse human preferences, highlighting the importance of usability and its effect on rapid adoption of LLMs.  │
│  The authors propose methods to democratize access and engagement with LLMs to improve alignment outcomes.      │
│     **Relevance**: This work matters as it emphasizes the societal impact of LLM alignment and strives to make  │
│  alignment accessible and equitable for a broader audience.                                                     │
│                                                                                                                 │
│  3. **Title**: [A Comprehensive Survey of LLM Alignment Techniques](https://arxiv.org/abs/2407.16216)           │
│     **Authors**: Z. Wang et al.                                                                                 │
│     **Year/Venue**: 2024, arXiv                                                                                 │
│     **Key Findings**: This survey categorizes the existing literature on LLM alignment into several key topics  │
│  and provides detailed elaborations on each method's strengths and weaknesses. The authors aim to bridge the    │
│  gaps in current research by synthesizing findings from various sources and guiding future research             │
│  directions.                                                                                                    │
│     **Relevance**: This paper is significant because i


RESEARCH RESULTS
1. **Title**: [Large Language Model Alignment: A Survey](https://arxiv.org/abs/2309.15025)  
   **Authors**: T. Shen et al.  
   **Year/Venue**: 2023, arXiv  
   **Key Findings**: This survey provides a comprehensive investigation of various alignment methodologies aimed at large language models (LLMs) and classifies them into distinct categories based on their theoretical frameworks and practical applications. The authors emphasize the significance of aligning LLMs with human values and decision-making processes.  
   **Relevance**: This paper is crucial as it offers an extensive overview of alignment techniques which is vital for understanding how to ensure LLMs act in accordance with human intentions.

2. **Title**: [Democratizing Large Language Model Alignment](https://proceedings.neurips.cc/paper_files/paper/2023/hash/949f0f8f32267d297c2d4e3ee10a2e7e-Abstract-Datasets_and_Benchmarks.html)  
   **Authors**: A. Köpf et al.  
   **Year/Venue**: 2023, NeurIPS  
   **